<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220914_%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LSTM(Long Short-Term Memory)

- 단순한 순환 신경망의 문제점
  - 시점이 흐를수록 지속해서 기억하지 못한다.
  - 그래디언트 손실 문제

- 이를 해결하기 위해 1997년 고안된 방벅
  - LSTM

- LSTM의 핵심은 정보를 여러 시점에 걸쳐 나르는 장치가 추가되었다는 것
  - 그래디언트를 보존할 수 있어 그래디언트 손실 문제가 발생하지 않도록 도와줌
  - 아래 그림에서 C로 표현된 Cell State h는 hidden state
  - 정보 나르는 것을 도와주는 3개의 게이트 forget, input, output

## 로이터 데이터셋
- reuters 데이터뎃
  - iMDB 데이터셋과 유사
  - 총 11258개의 뉴스기사가 46개 카테고리로 이루어져있음

데이터 다운로드 -> 문장 길이 동일하게 맞추기 -> 

In [ ]:
from tensorflow.keras.layers import LSTM, Flatten, Dense, Embedding
from tensorflow.keras.models import Sequential

model = Sequential()

model.add(Embedding(input_dim = num_words, output_dim = 64))
# SimpleRNN 층을 첫 번째 층으로 사용하는 경우,
# 반드시 input_shape를 명시해주어야 합니다.
# 새로운 인자 3개가 사용되었습니다.
# return_sequences, dropout, recurrent_dropout
model.add(LSTM(64, return_sequences =True, dropout = 0.15, input_dim = num_words
                    output_dim = 64, recurrent_dropout = 0.15))
model.add(LSTM(32))
model.add(Dense(46,activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['acc'])
model.summary()

history = model.fit(pad_X_train, y_train,
                    batch_size = 32, epochs = 15,
                    validation_split = 0.2)

## Conv1D
- 시퀀스 데이터도 컨볼루션 방법을 통해 특징을 추출할 수 있다.

- Conv1D층
  - 1차원 형태의 컨볼루션 필터를 가지는 1D 컨볼루션층
  - LSTM과 혼합하여 사용되는 등 순환 신경망과 함께 사용됨

- Conv2D: 2D 형태를 가진 컨볼루션 필터로 컨볼루션 연사늘 수행하여 이미지 특징을 추출
- Conv1D 1D 형태를 가진 컨볼루션 필터로 컨볼루션 연산을 수행하여 시퀀스 데이터의 특징을 추출

- 나는 좋다라는 문맥을 문장의 앞부분에서 인식했다면, 동일하게 뒷부분애서도 인식 할 수있음

In [1]:
from tensorflow.keras.datasets import imdb
# 1~10,000 빈도 순위에 해당하는 단어만 사용합니다.
num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 500

pad_X_train = pad_sequences(X_train,maxlen=max_len,)
pad_X_test = pad_sequences(X_test,maxlen=max_len,)

컨볼루션 1D층은 배치사이즈, 타임스텝, 채널) 형태를 입력으로 받으며, (배치사이즈,타임스텝,필터) 형태를 출력
- 컨볼루션 층ㅇ에서 특이하게 5 X 7 필터를 사용


In [4]:
from tensorflow.keras.layers import LSTM, Flatten, Dense, Embedding, Conv1D, MaxPooling1D,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential

model = Sequential()

model.add(Embedding(input_dim = num_words, output_dim = 32, input_length = max_len))

model.add(Conv1D(32, 7, activation = 'relu'))
model.add(MaxPooling1D(7))
model.add(Conv1D(32, 5, activation = 'relu'))
model.add(MaxPooling1D(5))
model.add(GlobalMaxPooling1D())
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
              metrics = ['acc'])
model.summary()

history = model.fit(pad_X_train, y_train,
                    batch_size = 32, epochs = 30,
                    validation_split = 0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           320000    
                                                                 
 conv1d_2 (Conv1D)           (None, 494, 32)           7200      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 70, 32)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 66, 32)            5152      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 13, 32)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)              